In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv("allpatents_with_abstracts_and_full_text.csv")

In [11]:
needed = df[["app_ref_doc_number", "classifications"]]

In [12]:
needed.loc[:,"classifications"] = needed["classifications"].apply(lambda x: [temp.strip()[1:-1] for temp in x[1:-1].split(",")])

In [19]:
# remove all rows with [''] in classifications
needed = needed[needed["classifications"].apply(lambda x: x != [''])]

In [20]:
# Add one hot encoding for top level classifications
top_level_classes = ["A", "B", "C", "D", "E", "F", "G", "H", "Y"]
# Only add one hot encoding if top level class is first letter of any of the strings in classifications
def top_level_class_presence(top_level):
    def internal_top_level_class_presence(x):
        for temp in x:
            if temp[0] == top_level:
                return 1
            break
        return 0
    return internal_top_level_class_presence
for top_level_class in top_level_classes:
    needed.loc[:,top_level_class] = needed["classifications"].apply(top_level_class_presence(top_level_class))   
needed.head()

/var/folders/5h/qrdvl60d00j1g6scrfzt8ggh0000gn/T/ipykernel_17516/1321958896.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needed.loc[:,top_level_class] = needed["classifications"].apply(top_level_class_presence(top_level_class))
/var/folders/5h/qrdvl60d00j1g6scrfzt8ggh0000gn/T/ipykernel_17516/1321958896.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needed.loc[:,top_level_class] = needed["classifications"].apply(top_level_class_presence(top_level_class))
/var/folders/5h/qrdvl60d00j1g6scrfzt8ggh

,app_ref_doc_number,classifications,A,B,C,D,E,F,G,H,Y
0,17520613,"[G02B 23/2484, H05K 1/14, H04N 5/2256, H04N 5/...",0,0,0,0,0,0,1,0,0
1,17758230,"[A61C 17/20, A61C 17/028, A61C 17/0208]",1,0,0,0,0,0,0,0,0
2,17400738,[A22C 17/00],1,0,0,0,0,0,0,0,0
3,17402065,"[G01S 7/497, G01S 7/4863, G01S 17/894]",0,0,0,0,0,0,1,0,0
4,17403730,"[G06T 13/40, G06T 7/73, G06N 3/08, G06T 2200/2...",0,0,0,0,0,0,1,0,0


In [21]:
second_level_classes = []
for temp in needed["classifications"]:
    for temp2 in temp:
        if temp2[0:3] not in second_level_classes:
            second_level_classes.append(temp2[0:3])
# Remove duplicates in second_level_classes
second_level_classes = list(set(second_level_classes))
second_level_classes = sorted(second_level_classes)


def second_level_class_presence(second_level):
    def internal_top_level_class_presence(x):
        for temp in x:
            if temp[0:3] == second_level:
                return 1
        return 0
    return internal_top_level_class_presence

new_columns = []
for second_level_class in second_level_classes:
    new_columns.append(pd.DataFrame({second_level_class:needed["classifications"].apply(second_level_class_presence(second_level_class))}))
# Add the new columns to the dataframe using concat
all_patents_with_second_level = pd.concat([needed,] + new_columns, axis=1)
all_patents_with_second_level.head()

,app_ref_doc_number,classifications,A,B,C,D,E,F,G,H,...,G21,H01,H02,H03,H04,H05,H10,Y02,Y04,Y10
0,17520613,"[G02B 23/2484, H05K 1/14, H04N 5/2256, H04N 5/...",0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,0
1,17758230,"[A61C 17/20, A61C 17/028, A61C 17/0208]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,17400738,[A22C 17/00],1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17402065,"[G01S 7/497, G01S 7/4863, G01S 17/894]",0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,17403730,"[G06T 13/40, G06T 7/73, G06N 3/08, G06T 2200/2...",0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Select all rows with top level classification and app_ref_doc_number
top_level_classes = ["A", "B", "C", "D", "E", "F", "G", "H", "Y"]
needed_top_level = all_patents_with_second_level[["app_ref_doc_number"] + top_level_classes]
needed_top_level.to_csv("top_level_classifications.csv", index=False)

In [24]:
# Select all columns except the top level classifications
needed_second_level = all_patents_with_second_level.drop(columns=top_level_classes)
# remove the classifications column
needed_second_level = needed_second_level.drop(columns=["classifications"])
needed_second_level.to_csv("second_level_classifications.csv", index=False)